# Setup e Checagens

Notebook para a configuração inicial de ambiente para execução de tarefas com Apache Spark. Etapas:
- Carrega as configurações do projeto;
- Define as seeds para garantir reprodutibilidade dos experimentos;
- Inicializa uma sessão Spark personalizada conforme os parâmetros definidos. 

Ao final, exibe uma amostra de dados gerados pelo Spark e encerra a sessão.

In [1]:
# --- Bootstrap (Colab + Local) com persistência no Drive e detecção da raiz do repo ---
import os, sys, subprocess, shutil
from pathlib import Path

GITHUB_USER = "silvaniacorreia"
REPO_NAME   = "ifood-case-cupons"
REPO_URL    = f"https://github.com/{GITHUB_USER}/{REPO_NAME}.git"

IN_COLAB = "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ
PERSIST_ON_DRIVE = True 

def run(cmd):
    print(">", " ".join(cmd))
    subprocess.check_call(cmd)

def find_project_root(start: Path = None) -> Path:
    """Sobe diretórios até achar requirements.txt e src/utils.py."""
    start = start or Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / "requirements.txt").exists() and (p / "src" / "utils.py").exists():
            return p
    return start 

if IN_COLAB:
    if PERSIST_ON_DRIVE:
        from google.colab import drive
        drive.mount("/content/drive", force_remount=False)

    CONTENT_DIR = Path("/content")
    PROJECT_DIR = CONTENT_DIR / REPO_NAME
    if not PROJECT_DIR.exists():
        run(["git", "clone", REPO_URL, str(PROJECT_DIR)])
    else:
        os.chdir(PROJECT_DIR)
        run(["git", "fetch", "--all"])
        run(["git", "checkout", "main"])
        run(["git", "pull", "--rebase", "origin", "main"])
    os.chdir(PROJECT_DIR)

    if PERSIST_ON_DRIVE:
        DRIVE_PROJ = Path("/content/drive/MyDrive") / REPO_NAME
        DRIVE_DATA = DRIVE_PROJ / "data"
        DRIVE_PROJ.mkdir(parents=True, exist_ok=True)
        DRIVE_DATA.mkdir(parents=True, exist_ok=True)
        LOCAL_DATA = PROJECT_DIR / "data"
        if not LOCAL_DATA.is_symlink():
            if LOCAL_DATA.exists() and any(LOCAL_DATA.iterdir()):
                for p in LOCAL_DATA.iterdir():
                    dst = DRIVE_DATA / p.name
                    if p.is_dir():
                        shutil.copytree(p, dst, dirs_exist_ok=True)
                    else:
                        shutil.copy2(p, dst)
                shutil.rmtree(LOCAL_DATA)
            os.symlink(str(DRIVE_DATA), str(LOCAL_DATA), target_is_directory=True)

    run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    run([sys.executable, "scripts/download_data.py"])

    if str(PROJECT_DIR) not in sys.path:
        sys.path.insert(0, str(PROJECT_DIR))
    print("✔️ Bootstrap concluído (Colab). Projeto:", PROJECT_DIR)

else:
    PROJECT_DIR = find_project_root(Path.cwd().resolve())
    os.chdir(PROJECT_DIR)  # <- garante que estamos na raiz do repo
    if str(PROJECT_DIR) not in sys.path:
        sys.path.insert(0, str(PROJECT_DIR))
    print("Execução local. Raiz do projeto:", PROJECT_DIR)


Execução local. Raiz do projeto: C:\Users\silva\OneDrive\Documentos\portfólio\ifood-case-cupons


In [2]:
# Download dos dados brutos
import sys, subprocess

print("Baixando/extraindo dados (scripts/download_data.py)...")
subprocess.check_call([sys.executable, "scripts/download_data.py"])
print("✔️ Dados prontos em data/raw/")

Baixando/extraindo dados (scripts/download_data.py)...
✔️ Dados prontos em data/raw/


In [3]:
# Teste do Spark
from src.utils import load_settings, set_seeds, get_spark, stop_spark

s = load_settings()                   # lê config (settings.yaml)
set_seeds(s.runtime.seed)             # fixa seeds para reprodutibilidade
spark = get_spark(
    app_name=s.runtime.spark.app_name,
    shuffle_partitions=s.runtime.spark.shuffle_partitions,
)
print("Spark OK:", spark.version)
spark.range(5).show()
stop_spark(spark)


Spark OK: 3.5.1
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

